# Data Analysis Nanodegree - Project 3 Report
**By: Juan Silva**

## Introduction

In this project we performed the tasks to aquire, audit, pre-process, import and analyze a data set from open street map.

The following was accomplished

1. Describe the set we are working with
2. Initial exploreation of the data set 
3. Audit data set for areas of concern prior to import
4. Devise strategies to fix the data based on our findings
5. Import the data into MondoDB for analysis (fixing the issues along the way)
6. Analyze the data to see where we could potentially have a better impact to contribute back to open street map

In this report we highlight the problems found and how they were attacked. We also do an overview of the data and present suggestion for improving it.

## Data Set

**Area Chosen:**
Ottawa, Ontario, Canada (Downtown)

**Open Street Map Link:**
http://www.openstreetmap.org/#map=13/45.4000/-75.6920

**File:**
ottawa.osm (83.3 MB)

**Motivations:**
This is the area where I live. I know the city very well and I think I can better understand and process the data that way. I am also very curious about analysing the data. Specially with regards to restaurants and amenities.


## Initial Exploration

The first thing is to load the file and do an initial audit looking for all the types of tags present in the file and their counts. There were 325528 Node elements and 57317 Way elememnts, which are the ones we wanted to import into MongoDB for analysis.

## Problems Encountered in Data

After some initial exploration the following problems were identified to be addressed:


1. Street names were abreviated (In English and In French). (i.e. "St." instead of "Street" or "Boul." instead of "Boulevard")
2. Postal Codes presented some problems like Province codes being prepended or extra characters like commas and other introduced. (i.e. "ON K1B 4V4", "K2C 0L5,")
3. In particular the 'Fee' tag of Parking amenities entries has a variety of values which made it inconsistent. (i.e. 'Yes', 'No', 'Customers only', 'interval', 'daily' etc..)

### Street Names

We first set a group of expected street term and then tred to find instances where the street does not follow the expected format. We used a regular expression to get the last word of the street and match against our expected list. Then we logged any entries not following the pattern.

After some iterations working out issues with language to match first words as opposed to last ones in French street names, the following entries were identified:

**English:** "Ave", "Blvd", "Dr.", "Rd"
**French:** "Boul."

These issues were addressed by creating a mapping dictionary that was used to replace appreviations in both languages with their corresponding full words.

### Postal Codes

After having imported data into MongoDB, some further exploration revealed problems with postal codes.
We used a regular expression to find any entries that didn't match the expected Canadian Postal Code pattern.

In [ ]:
#Check postal codes
pprint.pprint([doc for doc in db.ottawa.aggregate([
        {"$match":{"address.postcode": {"$not": re.compile("^[A-Za-z]\d[A-Za-z] ?\d[A-Za-z]\d$")}}}, 
        {"$group":{"_id":"$address.postcode", "count":{"$sum":1}}}, 
        {"$sort":{"count":-1}}
    ])])

Found these entries:

{u'K2A OE8', u'count': 2},  
{u'K2P 0P4,', u'count': 1},  
{u'JOX 2W1', u'count': 1},  
{u'K1N 7E7\u200e', u'count': 1},  
{u'ON K1N 9N8', u'count': 1}]

Since the data set was mostly clean and there were just a handful of exceptions, those were fixed by using a mapping dictionary and replacing the faulty entries with correct ones in the process of converting xml file into json format. Then re-import into the database.

### Fee Tag in Parking Amenities

When exploring the different values that the "Fee" tag in Parking amenities could take, we realized that the values were very different and not useful to do data aggregations when your primary concern is to know if you have to pay or not for a parking spot. 

Running a query like this:

In [ ]:
#Breakdown by kind of Fee related to parking
pprint.pprint([doc for doc in db.ottawa.aggregate([
                {"$match": {"amenity": "parking"}},
                {"$group": {"_id": "$fee", "Count": {"$sum": 1}}},
            ])])

Resulted in the following:

{u'Count': 1, u'_id': u'Daily fee'},  
{u'Count': 18, u'_id': u'no'},  
{u'Count': 1, u'_id': u'interval'},  
{u'Count': 65, u'_id': u'yes'},  
{u'Count': 1, u'_id': u'customer parking'},  
{u'Count': 1, u'_id': u'customers'}....  

So, in order to make this consistent we used a mapping dictionary to define for each of these values a "yes" or "no" in a new JSON attribute called "Paid". This was done in the process of converting the xml file into json format.

So when grouping by the new Paid field, the result was like:

{u'Count': 67, u'_id': u'yes'},  
{u'Count': 20, u'_id': u'no'}  



## Data Overview

We ran a series of queries against the DB to better understand the data set.
Our main goal is to identify areas of oportunity to contribute to the data set. For example, areas where data is missing or incomplete.

For this we ask a series of questions and run queries to answer them.

### How much are others contributing?
We wanted to know how much others contribute. So we list all contributors and their number of nodes authored.

In [ ]:
#Agregation, grouping by user and counting the elements (Output limmited to 100 for clarity in report)
pprint.pprint([doc for doc in db.ottawa.aggregate([
            {"$group": {"_id": "$created.user", "Contributed Items": {"$sum": 1}}},
            {"$project": {"_id": 0, "User": "$_id", "Contributed Items": 1}},
            {"$sort": {"Contributed Items": -1}},
            {"$limit": 3}
        ])
])

[{u'Contributed Items': 70539, u'User': u'andrewpmk'},  
 {u'Contributed Items': 56522, u'User': u'Johnwhelan'},  
 {u'Contributed Items': 51082, u'User': u'mcwetboy'},....  
 
 The average of elements contributed by user is : 797.6
 
 
### What amenities are available in this data set and how many of them?
This will give us an idea of the composition and kind of data that is available for this area

In [ ]:
#Aggregation grouping by amenity and counting entries
pprint.pprint([doc for doc in db.ottawa.aggregate([
            {"$match": {"amenity": {"$exists": 1}}},
            {"$group": {"_id": "$amenity", "Count": {"$sum": 1}}},
            {"$sort": {"Count": -1}}
        ])
])

The Top 3:

{u'Count': 2034, u'_id': u'parking'},  
{u'Count': 333, u'_id': u'restaurant'},  
{u'Count': 188, u'_id': u'school'},...  
 
Based on this results, we see that the more predominant ammenities by far are parking and restaurants. 

### What is being tracked for Parking and Restaurant amenities?

We wanted to understand what is being tracked about them so I can see where the areas of opportunity for contribution might be.

So, for both of them we want to see all the different Tags being recorded.

In [ ]:
#Map Reduce job to identify all the unique keys across all Parking documents
map = "function() { if(this.amenity == 'parking'){ for (var key in this) { emit(key, null); } } }"
reduce = "function(key, stuff) { return null; }"
result = db.ottawa.map_reduce(map, reduce, "myresults")
for doc in result.find():
    print doc

This results in a list of all keys. See the following sample:

...  
{u'_id': u'entrance', u'value': None}  
{u'_id': u'fee', u'value': None}  
{u'_id': u'highway', u'value': None}  
...  

From this list of things being tracked we have some areas of interest. Like knowing if a Fee is required, the kind of access defined for the parking and the type of parking (i.e. underground vs surface). For each of these we run queries that count the number of entries for each possible value.

As an example, here we check all the values by type of parking and how many entries were found:

In [69]:
#Breakdown by type of parking
pprint.pprint([doc for doc in db.ottawa.aggregate([
                {"$match": {"amenity": "parking"}},
                {"$group": {"_id": "$parking", "Count": {"$sum": 1}}},
            ])])

[{u'Count': 37, u'_id': u'multi-storey'},
 {u'Count': 43, u'_id': u'underground'},
 {u'Count': 1397, u'_id': u'surface'},
 {u'Count': 557, u'_id': None}]


Similar queries were ran for Restaurants. Areas of interest included "Cuisine", "Delivery" and "Wheelchair" accessibility.

## Findings and Ideas for Contribution

Based on the analysis we can see that the focus is very much on parking and restaurants. This makes sense and is probably among the things people would look for the most in a map.

We wanted to understand how these were being tracked and for that we identified all the tags for each on of those amenities. Then we analyzed how many values for some of those tags (of interest) were avilable.

It is clear that although there is some specific data avilable. A large number of entries come back with "None" as their value. Which means that it has not been defined yet for those entries. This indicates a clear area of opportunity to fill in the data.

To fill in those gaps we could envision different ways of doing it. 

One approach would be to leverage social media to ask people to contribute their knowledge about the amenities. Facebook apps with gamification involved could encourage people to browse some of the amenities and answer question where they know. This could be shared with friends to encourage others to contribute. This approach would have the benefit of leveraging the community knowledge aboutthe area. However, there could be problems with data input. Users could make mispellings, enter data in more than one way, or enter wrongful information. Some of these may be aliviated by providing as much as possible fixed groups of options like in drop down menus. It also poses the challenge of motivating people to actually participate and there is no guarantee of fulfilling all missing areas equally or by a specific timeline.

Another approach could be web data mining leveraging other sources that have compiled this information already. For example sites like yelp.ca could be parsed/queried to try to extract some of the missing information for those amenities. This will have the advantage of having full control of how much data we retrieve and the formatting of such data. We could identify the issues in the source data and do some munging to fit it to our model. Still if the source does not provide an API to obtain the information, we could face a lot of challenges to extract the information by scrapping the pages. Furthermore if the owners of the sites decide to change the markup of the page, our scrappers need to be updated. Then there is the issue with being banned by the sites if they have policies against crawlers going through their content. 

I think the approach to be taken will depend on the needs of the project that we may be working on which requires the data set to be more complete. Depending of the specifics of the information missing, one or other approach would be the right one.

